In [1]:
import fitz
import pandas as pd
from tqdm.auto import tqdm # thanh trạng thái

def text_formatter(text: str) -> str:
    cleaned_text = text.replace("\n", " ").strip()
    cleaned_text = cleaned_text.replace("÷"," đến ")
    cleaned_text= re.sub(r'\.+', '.', cleaned_text)
    cleaned_text = cleaned_text.replace(":"," ")
    cleaned_text = cleaned_text.replace("<","nhỏ hơn")
    cleaned_text = cleaned_text.replace("≥","lớn hơn bằng")
    cleaned_text = cleaned_text.replace(">","lớn hơn")
    cleaned_text = cleaned_text.replace("≤","nhỏ hơn bằng")
    return cleaned_text

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def tables_reader(tables):
    table_content = []
    for table in tables:
        if len(table[0]) >= 5:
            table = list(map(list, zip(*table)))
        titles = table.pop(0)
        table_content.append([])
        for i in range(len(table)):
            for j in range(len(table[i])):
                if table[i][j] == None:
                    table[i][j] = table[i-1][j]
                table[i][j] = titles[j] + " " + table[i][j]
            
            table_content[-1].append(" ".join(table[i]).replace('\n'," ")+". ")
    return table_content
                 

In [3]:
import pdfplumber

def extract_text_and_tables(pdf_path):
    text_segments = []
    # tables = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            
            if page.extract_tables():
                page_tables = page.extract_tables()
                tables_text = tables_reader(page_tables)
                for table in tables_text:
                    table_text = " ".join(table)+"."
                    text_segments.append(table_text)
            
            # Trích xuất văn bản
            page_text = page.extract_text()
            text_segments.append(page_text)

        text = " ".join(text_segments)
        return text

# text_segments = extract_text_and_tables(pdf_path)

# # Kiểm tra
# text_segments

In [4]:
seq_len = 400

In [5]:
import spacy
def split_chunk(input_list: list) -> list[list[str]]:
    slice_size = 2
    list_sentece = [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]
    chunks = []
    # print(slice_size)
    for chunk in list_sentece:            
        chunk = "".join(chunk).replace("  ", " ").strip()
        if len(chunk) > seq_len:
            nlp = spacy.blank("vi")
            nlp.add_pipe("sentencizer")
            chunk = list(nlp(chunk).sents)
            chunk = [str(sentence) for sentence in chunk]
            chunks.extend(chunk)
        else:
            chunks.append(chunk)
    return chunks

In [6]:
def sentences_split(texts):
    nlp = spacy.blank("vi")
    nlp.add_pipe("sentencizer") # Thêm pipeline sentencizer giúp phân đoạn câu
    texts = list(nlp(texts).sents) # gán vào pipeline
    texts = [str(sentence) for sentence in texts]
    for sentence in texts:
        if len(sentence) > seq_len:
            texts.remove(sentence)
            sentence = sentence.replace(";", ".")
            sentence = list(nlp(sentence).sents) # gán vào pipeline
            for item in sentence:
                texts.append(str(item))
    texts = split_chunk(texts)
    return texts

In [7]:
import re

# Tách văn bản quy định thành các phần dựa trên từ khóa 'Điều'. --> dễ tìm kiếm hơn
def split_regulation(pdf_path: str)-> list[dict]:

    pages_and_texts = []
    sections=[{'title':"",'content':""}]
    text = extract_text_and_tables(pdf_path)
    cur_main_sec = ""

    for line in text.split('\n'):
        # Tìm tiêu đề (dòng đầu tiên)
        title_match = re.match(r"(Điều\s\d+\.)", line)
        
        if not title_match:
            if cur_main_sec == "":
                sections[-1]['content'] += line + " "
            else:
                sections[-1]['content'] += " " + line   
        else:
            cur_main_sec = line
            sections.append({})
            sections[-1]['title'] = cur_main_sec
            sections[-1]['content'] = ""

    for section in sections:
            cleaned_content = text_formatter(section['content'])
            
            len_content = len(cleaned_content)
            
            if len_content > seq_len:
                cleaned_content = sentences_split(cleaned_content)
            else:
                cleaned_content = [cleaned_content]
            for i in range (len(cleaned_content)):
                pages_and_texts.append({"char_count": len(cleaned_content[i])+len(section['title']),
                                        "token_count": (len(cleaned_content[i])+len(section['title'])) / 4,  # 1 token = ~4 chars
                                        "text": f"{section['title']}\n {cleaned_content[i]}"})
    return pages_and_texts

In [ ]:
pdf_path = ""
pages_and_texts = split_regulation(pdf_path=pdf_path)

NameError: name 'pdf_path' is not defined

In [52]:
pages_and_texts_pd = pd.DataFrame(pages_and_texts)
pages_and_texts_pd

,char_count,token_count,text
0,0,0.00,\n
1,323,80.75,Điều 46. Quy định chuyển tiếp\n 1.Đối với các ...
2,33,8.25,Điều 46. Quy định chuyển tiếp\n 2.2.
3,31,7.75,Điều 46. Quy định chuyển tiếp\n 3.
4,491,122.75,Điều 46. Quy định chuyển tiếp\n Đối với các kh...
5,491,122.75,Điều 46. Quy định chuyển tiếp\n Đối với các kh...
6,463,115.75,Điều 46. Quy định chuyển tiếp\n Đối với các kh...


In [ ]:
def split_section(file_path):
    sections=[{'title':"",'content':""}]
    text = ""
    cur_main_sec = ""
    cur_sub_main_sec = ""
    cur_sub_sub_main_sec = ""
    pages_and_texts = []
    # Đọc nội dung từ file PDF
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
        
        # Tìm các mục bằng regex
        for line in text.split('\n'):
            section_match = re.match(r'^([A-Z]+\.)*((\d+\.)(\d+\.)?(\d+\.)?)',line)
            if not section_match:
                if cur_main_sec == "":
                    sections[-1]['content'] += line + " "
                else:
                    sections[-1]['content'] += " " + line
            elif section_match.group(1):
                cur_main_sec = line
                sections.append({})
                sections[-1]['title'] = cur_main_sec
                
                sections[-1]['content'] = ""
            elif section_match.group(5):
                cur_sub_sub_main_sec = cur_main_sec +" "+ line
                sections.append({})
                sections[-1]['title'] = cur_sub_sub_main_sec
                sections[-1]['content'] = ""
            elif section_match.group(4):
                cur_sub_main_sec = cur_main_sec +" "+ line
                sections.append({})
                sections[-1]['title'] = cur_sub_main_sec
                sections[-1]['content'] = ""
            elif section_match.group(3):
                cur_main_sec = line
                print(cur_main_sec)
                sections.append({})
                sections[-1]['title'] = cur_main_sec
                sections[-1]['content'] = ""
        for section in sections:
            cleaned_content = text_formatter(section['content'])
            len_content = len(cleaned_content)
            
            if len_content > seq_len:
                cleaned_content = sentences_split(cleaned_content)
            else:
                cleaned_content = [cleaned_content]
            for i in range (len(cleaned_content)):
                pages_and_texts.append({"char_count": len(cleaned_content[i])+len(section['title']),
                                        "token_count": (len(cleaned_content[i])+len(section['title'])) / 4,  # 1 token = ~4 chars
                                        "text": f"{section['title']}\n {cleaned_content[i]}"})
                
    return pages_and_texts

# Đọc file và hiển thị kết quả
file_path = "D:\CODE\DA2\Trợ lý ảo\Documents\Hướng dẫn\Phương pháp lập Kế hoạch học tập.pdf"
pages_and_texts = split_section(file_path)


1. Đối với các khoá tuyển sinh có quyết định công nhận NCS sau ngày 01 tháng
2. Đối với các khoá tuyển sinh có quyết định công nhận NCS trong khoảng thời
2. Đối với các khoá tuyển sinh đã có quyết định công nhận NCS trong khoảng
3. Đối với các khoá tuyển sinh đã có quyết định công nhận NCS trước ngày 18


In [32]:
pages_and_texts_pd = pd.DataFrame(pages_and_texts)
pages_and_texts_pd

,char_count,token_count,text
0,29,7.25,\n Điều 46. Quy định chuyển tiếp
1,294,73.50,1. Đối với các khoá tuyển sinh có quyết định c...
2,464,116.00,2. Đối với các khoá tuyển sinh có quyết định c...
3,436,109.00,2. Đối với các khoá tuyển sinh đã có quyết địn...
4,464,116.00,3. Đối với các khoá tuyển sinh đã có quyết địn...
